# HuSSPred Virtual Screening Instructions

This guide explains how to use the provided Jupyter Notebook script to run virtual screening predictions for skin sensitization using the **HuSSPred** model.

---

## 🚀 How to Use This Script

1. **Prepare Your Input File**  
   - Your input file must be an **Excel file** (`.xlsx` format).  
   - It must contain a column named: **"QSAR_READY_SMILES"** with SMILES representations of the chemicals.
   - It must be added to the **"../Batch_predictions/data"** Folder

2. **Modify the Script for Your File**  
   - Locate the following line in the script:
     ```python
     data_vs = "../data/FILENAME.xlsx"
     ```
   - Change only the **filename** at the end (`FILENAME.xlsx`) to match your actual Excel file.

3. **Run the Notebook**  
   - Open the Jupyter Notebook.  
   - Run all the cells to process your data and obtain predictions.

4. **Where to Find the Results**  
   - The predictions will be saved as a new Excel file at:  
     ```python
     output_path = "../results/Results_batch_predictions.xlsx"
     ```
   - The results file will contain:
     - **Binary Probability Active**: The probability of a compound being active.
     - **Binary Predicted Outcome**: `"1"` (active) or `"0"` (inactive).
     - **Binary Applicability Domain**: `"Inside"` (reliable prediction) or `"Outside"` (unreliable prediction).

---

## 📂 File Locations  
- **Input file**: Place your Excel file inside the `../data` folder.  
- **Output file**: The results will be saved in the `../results/` folder.  

Once you run the script, your predictions will appear in the output Excel file. 🚀

# Binary Prediction

In [5]:
# Importing packages
import os
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, PandasTools
import numpy as np
import pandas as pd
from pandas import DataFrame
import joblib
from sklearn.metrics import euclidean_distances

# Load test compounds from Excel file for virtual screening
data_vs = "../data/Sample_SMILES.xlsx"
output_path = "../results/Results_batch_predictions.xlsx"

test_df = pd.read_excel(data_vs)

# Standardize column names
test_df.columns = test_df.columns.str.strip().str.upper()  # Convert to uppercase for consistency

# Verify the correct column name for QSAR-ready SMILES
expected_col = "QSAR_READY_SMILES"
expected_col = expected_col.upper()  # Ensure uppercase matching

if expected_col not in test_df.columns:
    closest_match = [col for col in test_df.columns if expected_col in col.upper()]
    if closest_match:
        expected_col = closest_match[0]  # Use the closest match
        print(f"Warning: Using '{expected_col}' instead of 'QSAR_READY_SMILES'")
    else:
        raise KeyError(f"Column '{expected_col}' not found in test_df. Available columns: {test_df.columns.tolist()}")

# Convert "QSAR_READY_SMILES" to RDKit molecules, ensuring invalid SMILES are removed
valid_smiles = []
valid_mols = []

for smi in test_df[expected_col]:
    if pd.notna(smi):  # Ensure the value is not NaN
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
            valid_smiles.append(smi)
            valid_mols.append(mol)

# Ensure we only generate fingerprints for valid molecules
test_fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048, useFeatures=False) for m in valid_mols]

# Convert RDKit fingerprints to numpy array
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

# Convert fingerprints to numpy arrays
x_test = rdkit_numpy_convert(test_fp)

# Define dictionary with models and corresponding files
models_info = {
    "Overall": {
        "data_train": "../models/DT_overall_tox_sdf_final.sdf",
        "features": "selected_features_overall_tox_ecfp4.npy",
        "model": "ECFP4_overall_tox_svm.joblib",
        "ad": "overall_toxicity_AD.pkl"
    },
    "First Trimester": {
        "data_train": "../models/DT_first_trimester_tox_sdf_final.sdf",
        "features": "selected_features_first_tri_ecfp4.npy",
        "model": "ECFP4_first_trimester_svm.joblib",
        "ad": "first_tri_toxicity_AD.pkl"
    },
    "Second Trimester": {
        "data_train": "../models/DT_second_trimester_tox_sdf_final.sdf",
        "features": "selected_features_second_tri_ecfp4.npy",
        "model": "ECFP4_second_trimester_svm.joblib",
        "ad": "second_tri_toxicity_AD.pkl"
    },
    "Third Trimester": {
        "data_train": "../models/DT_third_trimester_tox_sdf_final.sdf",
        "features": "selected_features_third_tri_ecfp4.npy",
        "model": "ECFP4_third_trimester_svm.joblib",
        "ad": "third_tri_toxicity_AD.pkl"
    }
}

# Initialize results dataframe with the QSAR-ready SMILES for merging
results_df = DataFrame({"QSAR_READY_SMILES": valid_smiles})

# Define the applicability domain calculation function
def calculate_applicability_domain(X_train, X_test, threshold=0.5):
    distances = euclidean_distances(X_train, X_train)
    avg_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Define applicability domain threshold
    apd_threshold = avg_distance + threshold * std_distance
    
    # Calculate test distances to training data
    test_distances = euclidean_distances(X_test, X_train)
    min_distances = np.min(test_distances, axis=1)
    
    # Calculate applicability scores
    applicability_scores = 1 - (min_distances / apd_threshold)
    
    # Ensure the applicability scores are between 0 and 1
    applicability_scores = np.clip(applicability_scores, 0, 1)
    
    # Convert to percentage
    applicability_scores_percentage = applicability_scores * 100
    
    return applicability_scores_percentage

# Prediction loop for each model
for model_name, files in models_info.items():
    print(f"Processing: {model_name}")
    
    # Load training compounds
    train_mols = [mol for mol in Chem.SDMolSupplier(files["data_train"]) if mol is not None]
    
    # Generate binary Morgan fingerprint with radius 2 (no features) for training data
    train_fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048, useFeatures=False) for m in train_mols]
    
    # Convert training fingerprints to numpy array
    x_train = rdkit_numpy_convert(train_fp)
    
    # Load selected feature indices from the specified folder
    load_folder = "../models"
    load_path = os.path.join(load_folder, files["features"])
    selected_features = np.load(load_path)
    
    # Apply selected features
    x_train_selected = x_train[:, selected_features]
    x_test_selected = x_test[:, selected_features]
    
    # Load the calibrated SVM model and the threshold
    calibrated_model_data = joblib.load(os.path.join(load_folder, files["model"]))
    loaded_model = calibrated_model_data['model']
    loaded_threshold = calibrated_model_data['threshold']
    
    # Calculate applicability domain for test data
    applicability_scores = calculate_applicability_domain(x_train_selected, x_test_selected)
    
    # Check for any NaN values in the descriptors
    if np.isnan(x_test_selected).any():
        raise ValueError("NaN values found in descriptors.")
    
    # Predict new data
    orig_pp1 = loaded_model.predict_proba(x_test_selected)[:, 1]
    
    # Check for NaN values in the predictions
    if np.isnan(orig_pp1).any():
        raise ValueError("NaN values found in predictions.")
    
    # Apply the threshold to binarize predictions
    predicted_classes = (orig_pp1 >= loaded_threshold).astype(int)
    
    # Convert numpy array to pandas dataframe with SMILES column
    vs1 = DataFrame({
        "QSAR_READY_SMILES": valid_smiles,  # Ensure SMILES are included
        f" {model_name.upper()} BINARY PROBABILITY ACTIVE": np.round(orig_pp1, 2),
        f"{model_name.upper()} BINARY PREDICTED OUTCOME": np.where(orig_pp1 >= loaded_threshold, "1", "0"),
        f"{model_name.upper()} BINARY APPLICABILITY DOMAIN": np.where(applicability_scores >= 50, "Inside", "Outside")
    })
    
    # Ensure column names are properly formatted before merging
    vs1.columns = vs1.columns.str.strip().str.upper()
    
    # Merge predictions with the results dataframe while preserving all original QSAR-ready SMILES
    results_df = results_df.merge(vs1, on="QSAR_READY_SMILES", how="left")

# Merge predictions with the original dataset while preserving all original columns
if expected_col in test_df.columns and expected_col in results_df.columns:
    final_df = test_df.merge(results_df, on=expected_col, how="left")
else:
    raise KeyError(f"Column '{expected_col}' missing from one of the datasets. Test_df columns: {test_df.columns.tolist()}, results_df columns: {results_df.columns.tolist()}")

# Save the updated dataset with all original columns + virtual screening results
final_df.to_excel(output_path, sheet_name="Sheet1", index=False)

# Print final dataframe
print(final_df)

[00:53:48] SMILES Parse Error: syntax error while parsing: 
[00:53:48] SMILES Parse Error: Failed parsing SMILES ' ' for input: ' '
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use

Processing: Overall
Processing: First Trimester
Processing: Second Trimester
Processing: Third Trimester


[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerator
[00:53:48] DEPRECATION WARNING: please use MorganGenerat

                                      QSAR_READY_SMILES  \
0                          [O-][N+](=O)C1=CC=C(CBr)C=C1   
1                          [O-][N+](=O)C1=CC=C(CCl)C=C1   
2                                        BrCC1=CC=CC=C1   
3                                        O=CC1=CC=CC=C1   
4                                         NNC1=CC=CC=C1   
...                                                 ...   
5485         [H]C12CC([H])(C(C)C3(CCC(=O)C=C3)C1)C2(C)C   
5486                   [H]C12CC([H])(C(=C)C(O)C1)C2(C)C   
5487  [H]C1(CCC2C3([H])C(O)CC4CC(O)CCC4(C)C3CC(O)C12...   
5488                  [H]C(CCCC)=C([H])C([H])=C([H])C=O   
5489                     [H]C(C(=O)NC)=C(C)OP(=O)(OC)OC   

      OVERALL BINARY PROBABILITY ACTIVE OVERALL BINARY PREDICTED OUTCOME  \
0                                  0.37                                1   
1                                  0.45                                1   
2                                  0.10                        